# ETL (Extract, Transform, Load)

El proceso ETL es fundamental para preparar los datos de Siniestros Viales para su análisis. Al extraer, transformar y cargar los datos de manera efectiva, se mejora la calidad de la información, se facilita la comprensión de los patrones y tendencias, y se posibilita la toma de decisiones informada en cuestiones de seguridad vial. La normalización, limpieza y organización adecuadas de los datos son esenciales para garantizar la validez y utilidad de los resultados del análisis.

## 0. Importacion de librerias

In [754]:
import pandas as pd
import calendar
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

## 1. Extraccion de datos

In [755]:
df_hechos = pd.read_excel("Datasets\homicidios.xlsx", sheet_name="HECHOS")

In [756]:
df_victimas = pd.read_excel("Datasets\homicidios.xlsx", sheet_name="VICTIMAS")

## 2. Exploracion Inicial

### 2a. Examinamos la estructura de los dataframes

In [757]:
df_victimas.head(5)

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02 00:00:00
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03 00:00:00
3,2016-0004,2016-01-10,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18,SD
4,2016-0005,2016-01-21,2016,1,21,CONDUCTOR,MOTO,MASCULINO,29,2016-02-01 00:00:00


In [758]:
df_victimas.shape

(717, 10)

In [759]:
df_hechos.head(5)

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,2034.0,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,NaN,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,SD
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,...,NaN,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS


In [760]:
df_hechos.shape

(696, 21)

In [761]:
df_hechos.columns

Index(['ID', 'N_VICTIMAS', 'FECHA', 'AAAA', 'MM', 'DD', 'HORA', 'HH',
       'LUGAR_DEL_HECHO', 'TIPO_DE_CALLE', 'Calle', 'Altura', 'Cruce',
       'Dirección Normalizada', 'COMUNA', 'XY (CABA)', 'pos x', 'pos y',
       'PARTICIPANTES', 'VICTIMA', 'ACUSADO'],
      dtype='object')

### 2b. Verificamos los tipos de datos de cada columna

In [762]:
df_hechos.dtypes

ID                               object
N_VICTIMAS                        int64
FECHA                    datetime64[ns]
AAAA                              int64
MM                                int64
DD                                int64
HORA                             object
HH                               object
LUGAR_DEL_HECHO                  object
TIPO_DE_CALLE                    object
Calle                            object
Altura                          float64
Cruce                            object
Dirección Normalizada            object
COMUNA                            int64
XY (CABA)                        object
pos x                            object
pos y                            object
PARTICIPANTES                    object
VICTIMA                          object
ACUSADO                          object
dtype: object

In [763]:
df_victimas.dtypes

ID_hecho                       object
FECHA                  datetime64[ns]
AAAA                            int64
MM                              int64
DD                              int64
ROL                            object
VICTIMA                        object
SEXO                           object
EDAD                           object
FECHA_FALLECIMIENTO            object
dtype: object

### 2c. Comprobamos los datos nulos de cada variable

In [764]:
df_victimas.isnull().sum()

ID_hecho               0
FECHA                  0
AAAA                   0
MM                     0
DD                     0
ROL                    0
VICTIMA                0
SEXO                   0
EDAD                   0
FECHA_FALLECIMIENTO    0
dtype: int64

In [765]:
df_hechos.isnull().sum()

ID                         0
N_VICTIMAS                 0
FECHA                      0
AAAA                       0
MM                         0
DD                         0
HORA                       0
HH                         0
LUGAR_DEL_HECHO            0
TIPO_DE_CALLE              0
Calle                      1
Altura                   567
Cruce                    171
Dirección Normalizada      8
COMUNA                     0
XY (CABA)                  0
pos x                      0
pos y                      0
PARTICIPANTES              0
VICTIMA                    0
ACUSADO                    0
dtype: int64

### 2d. Examinamos datos duplicados

In [766]:
hechos_duplicados = df_hechos.duplicated().sum()
victimas_duplicados = df_victimas.duplicated().sum()
print("Cantidad de duplicados en df_hechos:", hechos_duplicados)
print("Cantidad de duplicados en df_victimas:", victimas_duplicados)

Cantidad de duplicados en df_hechos: 0
Cantidad de duplicados en df_victimas: 0


### Conclusiones preliminares:
* Datos de Víctimas:<br>
El DataFrame de víctimas proporciona información detallada sobre las personas involucradas en los siniestros.<br>
* Relación entre Variables de Hechos y Víctimas:<br>
Las variables compartidas entre los dos DataFrames, como el identificador del siniestro ("ID"), pueden usarse para vincular la información de hechos y víctimas. <br>
* Consistencia en la Integración de Datos:<br>
Aunque el DataFrame de hechos tiene 696 filas y el DataFrame de víctimas tiene 717 filas, se observa que hay más víctimas que hechos.<br>
* Datos Faltantes:<br>
Se identificaron valores faltantes en las columnas "Altura","Calle","Cruce" y "Dirección Normalizada". Estos requerirán un manejo adecuado durante el análisis.<br>
* Tipos de Datos:<br>
La columna "HORA" parece estar en formato de cadena. Se sugiere convertirla a un formato de tiempo. Además, la columna "EDAD" en el DataFrame de hechos tiene el tipo de dato "object," lo que podría beneficiarse de una conversión a un tipo numérico.<br>
* Geocodificación:<br>
Las columnas "pos x" y "pos y" proporcionan información de geolocalización. Se podran utilizar para visualizar la distribución espacial de los siniestros en un mapa. Estas variables tambien nos permiten mapear geográficamente los siniestros, identificando áreas de alta frecuencia.<br>
* Datos de Víctimas:<br>
Las variables relacionadas con las víctimas, como "SEXO" y "EDAD," ofrecen información valiosa sobre el perfil de las personas involucradas en los siniestros.<br>
* Datos de Participantes:<br>
La variable "PARTICIPANTES" indica siniestros con múltiples vehículos involucrados.<br>


## 3. Limpieza y Transformacion de datos

### 3a. Renombrar columnas

In [767]:
df_hechos.columns

Index(['ID', 'N_VICTIMAS', 'FECHA', 'AAAA', 'MM', 'DD', 'HORA', 'HH',
       'LUGAR_DEL_HECHO', 'TIPO_DE_CALLE', 'Calle', 'Altura', 'Cruce',
       'Dirección Normalizada', 'COMUNA', 'XY (CABA)', 'pos x', 'pos y',
       'PARTICIPANTES', 'VICTIMA', 'ACUSADO'],
      dtype='object')

In [768]:
df_hechos.rename(columns={"ID":"Id",
                          "N_VICTIMAS":"Nro Victimas",
                          "FECHA":"Fecha",
                          "AAAA":"Año",
                          "MM":"Mes",
                          "DD":"Dia",
                          "HORA":"Hora",
                          "HH":"Franja Horaria",
                          "LUGAR_DEL_HECHO":"Lugar",
                          "TIPO_DE_CALLE": "Tipo Calle",
                          "COMUNA":"Comuna",
                          "XY (CABA)":"Geocodificacion",
                          "pos x":"Longitud",
                          "pos y":"Latitud",
                          "PARTICIPANTES":"Participantes",
                          "VICTIMA":"Victima",
                          "ACUSADO":"Acusado"}, inplace=True)

In [769]:
df_victimas.columns

Index(['ID_hecho', 'FECHA', 'AAAA', 'MM', 'DD', 'ROL', 'VICTIMA', 'SEXO',
       'EDAD', 'FECHA_FALLECIMIENTO'],
      dtype='object')

In [770]:
df_victimas.rename(columns={"ID_hecho":"Id",
                          "FECHA":"Fecha",
                          "AAAA":"Año",
                          "MM":"Mes",
                          "DD":"Dia",
                          "ROL":"Rol",
                          "VICTIMA":"Victima",
                          "SEXO":"Sexo",
                          "EDAD": "Edad",
                          "FECHA_FALLECIMIENTO":"Fecha_fallecimiento"}, inplace=True)

### 3b. Manejo de valores nulos 

In [771]:
df_hechos[["Altura", "Calle", "Cruce","Dirección Normalizada"]].isnull().sum()

Altura                   567
Calle                      1
Cruce                    171
Dirección Normalizada      8
dtype: int64

In [772]:
df_hechos[["Altura", "Calle", "Cruce","Dirección Normalizada"]].isnull().mean()*100

Altura                   81.465517
Calle                     0.143678
Cruce                    24.568966
Dirección Normalizada     1.149425
dtype: float64

En el caso de la variable"Calle" las filas con nulos representan una proporcion muy baja en cuanto al total de registros por lo que se procede a eliminar.

In [773]:
df_hechos = df_hechos.dropna(subset=["Calle"])
df_hechos_calle_nulos = df_hechos["Calle"].isnull().sum()
print(df_hechos_calle_nulos)

0


La variable "Altura" indica el punto de la calle donde ocurrio el hecho, esta variable tiene la gran mayoria de los registros nulos por lo que no nos aportar informacion relevante para el analisis por lo que se procede a eliminar la misma. Tambien hay que tener en cuenta que gran parte de los hechos ocurren en intersecciones y cuando esto no sucede la altura de la calle suele estar especificada en la variable "Lugar del hecho"

In [774]:
df_hechos = df_hechos.drop(["Altura"], axis=1)
df_hechos.head(5)

,Id,Nro Victimas,Fecha,Año,Mes,Dia,Hora,Franja Horaria,Lugar,Tipo Calle,Calle,Cruce,Dirección Normalizada,Comuna,Geocodificacion,Longitud,Latitud,Participantes,Victima,Acusado
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,PIEDRA BUENA AV.,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,"PAZ, GRAL. AV.",DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,ENTRE RIOS AV.,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,LARRAZABAL AV.,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,SD
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,SAN JUAN AV.,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS


En cuanto a la variable "Cruce" vamos a verificar si la falta de valores en la columna se justifica por la naturaleza del hecho (ocurrido en un cruce o no)

In [775]:
con_cruce = df_hechos[df_hechos["Cruce"].notna()]
contexto = con_cruce[["Lugar","Tipo Calle","Cruce"]]

In [776]:
contexto.head()

,Lugar,Tipo Calle,Cruce
0,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,"FERNANDEZ DE LA CRUZ, F., GRAL. AV."
1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,DE LOS CORRALES AV.
3,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,"VILLEGAS, CONRADO, GRAL."
4,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,"SAENZ PE?A, LUIS, PRES."
5,AV 27 DE FEBRERO Y AV ESCALADA,AVENIDA,ESCALADA AV.


In [777]:
sin_cruce = df_hechos[df_hechos["Cruce"].isna()]
contexto_sin_cruce = sin_cruce[["Lugar","Tipo Calle"]]

In [778]:
contexto_sin_cruce.head()

,Lugar,Tipo Calle
2,AV ENTRE RIOS 2034,AVENIDA
9,AV ENTRE RIOS 1366,AVENIDA
14,SUIPACHA 156,CALLE
33,LIMA 1483,CALLE
35,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI KM....,AUTOPISTA


En base a lo observado arriba podemos asumir que los valores nulos que se encuentran en la variable corresponden a hechos que no sucedieron en un cruce por lo que procedemos a imputar esos valores como "Sin cruce"

In [779]:
df_hechos["Cruce"].fillna("Sin cruce", inplace=True)
df_hechos_nulos = df_hechos["Cruce"].isnull().sum()
print("Cantidad de valores nulos la variable Cruce:", df_hechos_nulos)

Cantidad de valores nulos la variable Cruce: 0


La variable "Direccion normalizada" representa a la calle donde ocurrio el hecho, en este caso la cantidad de valores nulos son pocos en comparacion con el total de registros por lo que se procede a eliminar los nulos

In [780]:
df_hechos = df_hechos.dropna(subset=["Dirección Normalizada"])
df_hechos_direccion_nulos = df_hechos["Dirección Normalizada"].isnull().sum()
print("Cantidad de valores nulos la variable Direccion Normalizada:", df_hechos_direccion_nulos)

Cantidad de valores nulos la variable Direccion Normalizada: 0


### 3b. Conversion del tipo de datos y manejo de datos faltantes

#### Dataframe HECHOS

Verificamos los tipos de datos que tenemos de manera general

In [781]:
df_hechos.dtypes

Id                               object
Nro Victimas                      int64
Fecha                    datetime64[ns]
Año                               int64
Mes                               int64
Dia                               int64
Hora                             object
Franja Horaria                   object
Lugar                            object
Tipo Calle                       object
Calle                            object
Cruce                            object
Dirección Normalizada            object
Comuna                            int64
Geocodificacion                  object
Longitud                         object
Latitud                          object
Participantes                    object
Victima                          object
Acusado                          object
dtype: object

Como lo indica nuestro diccionario los valores en los cuales no se tiene registro se coloca SD que indica Sin Dato por lo que se verifica que columnas tienen este tipo de registro y que cantidad

In [782]:
columnas_sd_dicc = {}  

for columna in df_hechos.columns:
    cantidad = (df_hechos[columna] == 'SD').sum()
    columnas_sd_dicc[columna] = cantidad

for columna, conteo in columnas_sd_dicc.items():
    print(f"Columna '{columna}': {conteo} veces que aparece 'SD'")

Columna 'Id': 0 veces que aparece 'SD'
Columna 'Nro Victimas': 0 veces que aparece 'SD'
Columna 'Fecha': 0 veces que aparece 'SD'
Columna 'Año': 0 veces que aparece 'SD'
Columna 'Mes': 0 veces que aparece 'SD'
Columna 'Dia': 0 veces que aparece 'SD'
Columna 'Hora': 1 veces que aparece 'SD'
Columna 'Franja Horaria': 1 veces que aparece 'SD'
Columna 'Lugar': 0 veces que aparece 'SD'
Columna 'Tipo Calle': 0 veces que aparece 'SD'
Columna 'Calle': 0 veces que aparece 'SD'
Columna 'Cruce': 0 veces que aparece 'SD'
Columna 'Dirección Normalizada': 0 veces que aparece 'SD'
Columna 'Comuna': 0 veces que aparece 'SD'
Columna 'Geocodificacion': 0 veces que aparece 'SD'
Columna 'Longitud': 0 veces que aparece 'SD'
Columna 'Latitud': 0 veces que aparece 'SD'
Columna 'Participantes': 0 veces que aparece 'SD'
Columna 'Victima': 9 veces que aparece 'SD'
Columna 'Acusado': 21 veces que aparece 'SD'


Aqui verificamos los tipos de datos que contienen las variables en las cuales estamos interesados en cambiar el tipo

### Fecha

In [783]:
df_hechos["Fecha"].apply(type).value_counts()

Fecha
<class 'pandas._libs.tslibs.timestamps.Timestamp'>    688
Name: count, dtype: int64

En la variable fecha eliminamos la estampa de tiempo para quedarnos con la fecha solamente

In [784]:
df_hechos["Fecha"]= df_hechos["Fecha"].dt.date

In [785]:
df_hechos["Fecha"].apply(type).value_counts()

Fecha
<class 'datetime.date'>    688
Name: count, dtype: int64

### Hora

In [786]:
df_hechos["Hora"].apply(type).value_counts()

Hora
<class 'datetime.time'>        600
<class 'str'>                   85
<class 'datetime.datetime'>      3
Name: count, dtype: int64

In [787]:
df_hechos["Hora"].unique()[:10]

array([datetime.time(4, 0), datetime.time(1, 15), datetime.time(7, 0),
       datetime.time(0, 0), datetime.time(5, 20), datetime.time(18, 30),
       datetime.time(19, 10), datetime.time(15, 20), datetime.time(1, 20),
       datetime.time(11, 30)], dtype=object)

In [788]:
def convertir_a_time(valor):
    try:
        return datetime.strptime(str(valor), '%H:%M:%S').time()
    except (ValueError, TypeError):
        return None

Convertimos el tipo de dato Hora de object a time

In [789]:
df_hechos["Hora"] = df_hechos["Hora"].apply(convertir_a_time)
df_hechos["Hora"].apply(type).value_counts()

Hora
<class 'datetime.time'>    684
<class 'NoneType'>           4
Name: count, dtype: int64

Verificamos que hay cuatro valores nulos dentro de Hora, al ser pocos valores en comparacion con el total de registros se decidde imputar los valores con la hora mas comun

In [790]:
hora_moda = df_hechos["Hora"].mode()[0]
df_hechos["Hora"].fillna(hora_moda, inplace=True)
df_hechos["Hora"].apply(type).value_counts()

Hora
<class 'datetime.time'>    688
Name: count, dtype: int64

### Franja Horaria

In [791]:
df_hechos["Franja Horaria"].apply(type).value_counts()

Franja Horaria
<class 'int'>    687
<class 'str'>      1
Name: count, dtype: int64

In [792]:
df_hechos["Franja Horaria"].unique()

array([4, 1, 7, 0, 5, 18, 19, 15, 11, 22, 16, 9, 23, 6, 10, 17, 12, 8, 21,
       20, 14, 3, 2, 13, 'SD'], dtype=object)

En el caso de la variable HORA que representa la hora del siniestro se convierte el tipo de dato de object a datetime para su posterior analisis, pero en cuanto a la variable Franja Horaria que representa la franja horaria del siniestro se procede a conventir a tipo int pero previamente en el unico registro que se encuentra SD se procede a imputar el valor 

In [793]:
franja_hora_moda = df_hechos["Franja Horaria"].mode()[0]
df_hechos["Franja Horaria"] = df_hechos["Franja Horaria"].replace("SD",franja_hora_moda)

In [794]:
df_hechos["Franja Horaria"].unique()

array([ 4,  1,  7,  0,  5, 18, 19, 15, 11, 22, 16,  9, 23,  6, 10, 17, 12,
        8, 21, 20, 14,  3,  2, 13], dtype=int64)

In [795]:
df_hechos["Franja Horaria"].apply(type).value_counts()

Franja Horaria
<class 'int'>    688
Name: count, dtype: int64

### Tipo Calle, Victima, Comuna y Acusado

Convertimos las variables Tipo Calle, Victima, Comuna y Acusado de tipo object a category, dado que las mismas son variables categoricas

In [796]:
df_hechos["Tipo Calle"] = df_hechos["Tipo Calle"].astype("category")
df_hechos["Victima"] = df_hechos["Victima"].astype("category")
df_hechos["Acusado"] = df_hechos["Acusado"].astype("category")
df_hechos["Comuna"] = df_hechos["Comuna"].astype("category")

In [797]:
columnas_interesantes = ["Tipo Calle", "Victima", "Acusado","Comuna"]
tipos_de_datos = df_hechos[columnas_interesantes].dtypes
print(tipos_de_datos)

Tipo Calle    category
Victima       category
Acusado       category
Comuna        category
dtype: object


Para la variable Tipo Calle verificamos que hay una categoria que se denomina Gral Paz que se transformara a Autopista para que en el posterior analisis sea mas homogeneo

In [798]:
df_hechos["Tipo Calle"].unique()

['AVENIDA', 'GRAL PAZ', 'CALLE', 'AUTOPISTA']
Categories (4, object): ['AUTOPISTA', 'AVENIDA', 'CALLE', 'GRAL PAZ']

In [799]:
df_hechos["Tipo Calle"] = df_hechos["Tipo Calle"].replace({"GRAL PAZ": "AUTOPISTA"})
df_hechos.head()

,Id,Nro Victimas,Fecha,Año,Mes,Dia,Hora,Franja Horaria,Lugar,Tipo Calle,Calle,Cruce,Dirección Normalizada,Comuna,Geocodificacion,Longitud,Latitud,Participantes,Victima,Acusado
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,PIEDRA BUENA AV.,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,AUTOPISTA,"PAZ, GRAL. AV.",DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,ENTRE RIOS AV.,Sin cruce,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,LARRAZABAL AV.,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,SD
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,SAN JUAN AV.,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS


Las variables Victima y Acusado presentan registros SD es decir sin dato asi que procedemos a analizar

In [800]:
df_hechos["Victima"].unique()

['MOTO', 'AUTO', 'PEATON', 'SD', 'CARGAS', 'BICICLETA', 'PASAJEROS', 'MOVIL', 'OBJETO FIJO', 'PEATON_MOTO']
Categories (10, object): ['AUTO', 'BICICLETA', 'CARGAS', 'MOTO', ..., 'PASAJEROS', 'PEATON', 'PEATON_MOTO', 'SD']

Verificando el diccionario de datos vemos que hay categorias que no se encuentran en el diccionario por lo que vamos a proceder a agregarlos a las categoria otro

In [801]:
if "Victima" in df_hechos.columns:
    df_hechos["Victima"] = df_hechos["Victima"].replace({"OBJETO FIJO": "OTRO", "PEATON_MOTO": "OTRO"})
    print(df_hechos.head())
else:
    print("La columna 'Victima' no existe en el DataFrame.")

          Id  Nro Victimas       Fecha   Año  Mes  Dia      Hora  \
0  2016-0001             1  2016-01-01  2016    1    1  04:00:00   
1  2016-0002             1  2016-01-02  2016    1    2  01:15:00   
2  2016-0003             1  2016-01-03  2016    1    3  07:00:00   
3  2016-0004             1  2016-01-10  2016    1   10  00:00:00   
4  2016-0005             1  2016-01-21  2016    1   21  05:20:00   

   Franja Horaria                                      Lugar Tipo Calle  \
0               4  AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ    AVENIDA   
1               1          AV GRAL PAZ  Y AV DE LOS CORRALES  AUTOPISTA   
2               7                         AV ENTRE RIOS 2034    AVENIDA   
3               0      AV LARRAZABAL Y GRAL VILLEGAS CONRADO    AVENIDA   
4               5  AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA     AVENIDA   

              Calle                                Cruce  \
0  PIEDRA BUENA AV.  FERNANDEZ DE LA CRUZ, F., GRAL. AV.   
1    PAZ, GRAL. AV. 

In [802]:
df_hechos["Acusado"].unique()

['AUTO', 'PASAJEROS', 'SD', 'OBJETO FIJO', 'CARGAS', 'MOTO', 'MULTIPLE', 'OTRO', 'BICICLETA', 'TREN']
Categories (10, object): ['AUTO', 'BICICLETA', 'CARGAS', 'MOTO', ..., 'OTRO', 'PASAJEROS', 'SD', 'TREN']

Por ahora para ambas variables se decide conservar los valores SD dado que no representan un porcentaje grande dentro de cada variable

### Latitud y Longitud

En cuanto a las variables "Longitud" y "Latitud" represetan la longitud y latitud del lugar donde ocurrio el siniestro, por lo que se proce a transformar de object a numeric 

In [803]:
df_hechos["Latitud"].apply(type).value_counts()

Latitud
<class 'str'>    688
Name: count, dtype: int64

In [804]:
df_hechos["Longitud"].apply(type).value_counts()

Longitud
<class 'str'>    688
Name: count, dtype: int64

In [805]:
df_hechos["Longitud"] = pd.to_numeric(df_hechos["Longitud"], errors="coerce")
df_hechos["Latitud"] = pd.to_numeric(df_hechos["Latitud"], errors="coerce")

In [806]:
print(df_hechos["Latitud"].apply(type).value_counts())
print(df_hechos["Longitud"].apply(type).value_counts())

Latitud
<class 'float'>    688
Name: count, dtype: int64
Longitud
<class 'float'>    688
Name: count, dtype: int64


In [807]:
df_hechos["Latitud"].isnull().sum()

4

In [808]:
df_hechos["Longitud"].isnull().sum()

4

In [809]:
coordenadas_nulas = df_hechos[df_hechos["Latitud"].isnull() & df_hechos["Longitud"].isnull()]
coordenadas_nulas

,Id,Nro Victimas,Fecha,Año,Mes,Dia,Hora,Franja Horaria,Lugar,Tipo Calle,Calle,Cruce,Dirección Normalizada,Comuna,Geocodificacion,Longitud,Latitud,Participantes,Victima,Acusado
139,2016-0174,1,2016-12-27,2016,12,27,00:00:00,0,AUTOPISTA 25 DE MAYO,AUTOPISTA,AUTOPISTA 25 DE MAYO,Sin cruce,AUTOPISTA 25 DE MAYO,0,Point (. .),NaN,NaN,SD-SD,SD,SD
176,2017-0042,1,2017-04-10,2017,4,10,09:00:00,9,AV. LEOPOLDO LUGONES PKM 6900,AUTOPISTA,"LUGONES, LEOPOLDO AV.",Sin cruce,"LUGONES, LEOPOLDO AV.",14,Point (. .),NaN,NaN,MOTO-CARGAS,MOTO,CARGAS
256,2017-0140,1,2017-11-19,2017,11,19,23:22:17,23,AU ARTURO FRONDIZI PKM 3100,AUTOPISTA,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,Sin cruce,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,4,Point (. .),NaN,NaN,MOTO-PASAJEROS,MOTO,PASAJEROS
559,2020-0039,1,2020-09-01,2020,9,1,19:17:42,19,MURGUIONDO 2700,CALLE,MURGUIONDO,Sin cruce,MURGUIONDO,9,Point (. .),NaN,NaN,PEATON-CARGAS,PEATON,CARGAS


En este caso debido a la faltante de datos del primer registro se procede a eliminar pero los demas registros se rellenan con las coordenadas correspondientes

In [810]:
indice_a_eliminar = 139 
df_hechos = df_hechos.drop(indice_a_eliminar)

In [811]:
ubicaciones = {
    "AV. LEOPOLDO LUGONES PKM 6900": (-34.54843731715158, -58.43855324867025),
    "AU ARTURO FRONDIZI PKM 3100": (-34.62331225201537, -58.379564214248965),
    "MURGUIONDO 2700": (-34.6605132179536, -58.49294436801899),
}

for ubicacion, (latitud, longitud) in ubicaciones.items():
    df_hechos.loc[df_hechos["Lugar"] == ubicacion, "Latitud"] = latitud
    df_hechos.loc[df_hechos["Lugar"] == ubicacion, "Longitud"] = longitud

In [812]:
coordenadas_nulas2 = df_hechos[df_hechos["Latitud"].isnull() & df_hechos["Longitud"].isnull()]
coordenadas_nulas2

,Id,Nro Victimas,Fecha,Año,Mes,Dia,Hora,Franja Horaria,Lugar,Tipo Calle,Calle,Cruce,Dirección Normalizada,Comuna,Geocodificacion,Longitud,Latitud,Participantes,Victima,Acusado


### Geocodificacion

Esta variable se procedera a eliminar dado que ya se cuenta con la informacion pertinente a esta en las columnas Latitud y Longitud

In [813]:
df_hechos.drop(columns=["Geocodificacion"], inplace=True)

In [814]:
df_hechos.head(2)

,Id,Nro Victimas,Fecha,Año,Mes,Dia,Hora,Franja Horaria,Lugar,Tipo Calle,Calle,Cruce,Dirección Normalizada,Comuna,Longitud,Latitud,Participantes,Victima,Acusado
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,PIEDRA BUENA AV.,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,-58.475340,-34.687570,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,AUTOPISTA,"PAZ, GRAL. AV.",DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,-58.508775,-34.669777,AUTO-PASAJEROS,AUTO,PASAJEROS


#### Dataframe VICTIMAS

In [815]:
df_victimas.dtypes

Id                             object
Fecha                  datetime64[ns]
Año                             int64
Mes                             int64
Dia                             int64
Rol                            object
Victima                        object
Sexo                           object
Edad                           object
Fecha_fallecimiento            object
dtype: object

In [816]:
columnas_sd_dicc2 = {}  

for columna2 in df_victimas.columns:
    cantidad2 = (df_victimas[columna2] == 'SD').sum()
    columnas_sd_dicc2[columna2] = cantidad2

for columna2, conteo2 in columnas_sd_dicc2.items():
    print(f"Columna '{columna2}': {conteo2} veces que aparece 'SD'")

Columna 'Id': 0 veces que aparece 'SD'
Columna 'Fecha': 0 veces que aparece 'SD'
Columna 'Año': 0 veces que aparece 'SD'
Columna 'Mes': 0 veces que aparece 'SD'
Columna 'Dia': 0 veces que aparece 'SD'
Columna 'Rol': 11 veces que aparece 'SD'
Columna 'Victima': 9 veces que aparece 'SD'
Columna 'Sexo': 6 veces que aparece 'SD'
Columna 'Edad': 53 veces que aparece 'SD'
Columna 'Fecha_fallecimiento': 68 veces que aparece 'SD'


Buscamos si hay filas donde todos los registros sean SD

In [817]:
filas_sd = df_victimas[df_victimas.eq("SD").all(axis=1)]
filas_sd

,Id,Fecha,Año,Mes,Dia,Rol,Victima,Sexo,Edad,Fecha_fallecimiento


Verificamos el tipo de dato que poseen las variables que deseamos transformar

### Edad

In [818]:
df_victimas["Edad"].apply(type).value_counts()

Edad
<class 'int'>    664
<class 'str'>     53
Name: count, dtype: int64

In [819]:
df_victimas["Edad"].unique()

array([19, 70, 30, 18, 29, 22, 16, 59, 65, 34, 41, 50, 38, 21, 52, 36, 20,
       54, 'SD', 56, 24, 78, 79, 26, 57, 37, 58, 23, 60, 42, 53, 51, 40,
       87, 76, 75, 35, 80, 43, 45, 67, 27, 55, 49, 81, 25, 33, 46, 83, 39,
       28, 7, 48, 4, 82, 32, 17, 47, 61, 10, 95, 73, 84, 66, 85, 1, 15,
       13, 77, 44, 31, 62, 74, 71, 11, 86, 69, 72, 12, 63, 92, 68, 91, 64,
       5, 88], dtype=object)

Para la variable podemos observar que la mayoria de los registros se encuentran en el formato correcto pero algunos tienen formato string por lo que se procede a corregir

In [820]:
df_victimas["Edad"] = pd.to_numeric(df_victimas["Edad"], errors='coerce')
df_victimas["Edad"] = df_victimas.groupby("Sexo")["Edad"].transform(lambda x: x.fillna(x.median()))
tipo_edad = df_victimas["Edad"].apply(type).value_counts()
print("Tipos de datos de la variable:",tipo_edad)

Tipos de datos de la variable: Edad
<class 'float'>    717
Name: count, dtype: int64


### Fecha fallecimiento

In [821]:
df_victimas["Fecha_fallecimiento"].apply(type).value_counts()

Fecha_fallecimiento
<class 'datetime.datetime'>    648
<class 'str'>                   69
Name: count, dtype: int64

In [822]:
df_victimas["Fecha_fallecimiento"].unique()[:10]

array([datetime.datetime(2016, 1, 1, 0, 0),
       datetime.datetime(2016, 1, 2, 0, 0),
       datetime.datetime(2016, 1, 3, 0, 0), 'SD',
       datetime.datetime(2016, 2, 1, 0, 0),
       datetime.datetime(2016, 1, 24, 0, 0),
       datetime.datetime(2016, 1, 26, 0, 0),
       datetime.datetime(2016, 1, 29, 0, 0),
       datetime.datetime(2016, 2, 8, 0, 0),
       datetime.datetime(2016, 2, 10, 0, 0)], dtype=object)

Para dicha variable no se realizara la trasnformacion debido a que no sera necesaria para el posterior analisis del caso, por lo que procederemos a eliminarla

In [823]:
df_victimas= df_victimas.drop(columns=["Fecha_fallecimiento"], axis=1)

In [824]:
df_victimas.head(2)

,Id,Fecha,Año,Mes,Dia,Rol,Victima,Sexo,Edad
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19.0
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70.0


Comprobamos que se hayan realizado los cambios de manera correcta

In [825]:
columnas_sd_dicc2 = {}  

for columna2 in df_victimas.columns:
    cantidad2 = (df_victimas[columna2] == 'SD').sum()
    columnas_sd_dicc2[columna2] = cantidad2

for columna2, conteo2 in columnas_sd_dicc2.items():
    print(f"Columna '{columna2}': {conteo2} veces que aparece 'SD'")

Columna 'Id': 0 veces que aparece 'SD'
Columna 'Fecha': 0 veces que aparece 'SD'
Columna 'Año': 0 veces que aparece 'SD'
Columna 'Mes': 0 veces que aparece 'SD'
Columna 'Dia': 0 veces que aparece 'SD'
Columna 'Rol': 11 veces que aparece 'SD'
Columna 'Victima': 9 veces que aparece 'SD'
Columna 'Sexo': 6 veces que aparece 'SD'
Columna 'Edad': 0 veces que aparece 'SD'


### Rol

Al comprobarse que son pocos los datos donde falta la información de la variable "Rol", se decide imputar el valor mas frecuente para cada columna.

In [826]:
moda_rol = df_victimas["Rol"].mode()[0]
df_victimas.loc[df_victimas["Rol"] == "SD", "Rol"] = moda_rol

Comprobamos que se hayan realizado los cambios de manera correcta

In [827]:
columnas_sd_dicc2 = {}  

for columna2 in df_victimas.columns:
    cantidad2 = (df_victimas[columna2] == 'SD').sum()
    columnas_sd_dicc2[columna2] = cantidad2

for columna2, conteo2 in columnas_sd_dicc2.items():
    print(f"Columna '{columna2}': {conteo2} veces que aparece 'SD'")

Columna 'Id': 0 veces que aparece 'SD'
Columna 'Fecha': 0 veces que aparece 'SD'
Columna 'Año': 0 veces que aparece 'SD'
Columna 'Mes': 0 veces que aparece 'SD'
Columna 'Dia': 0 veces que aparece 'SD'
Columna 'Rol': 0 veces que aparece 'SD'
Columna 'Victima': 9 veces que aparece 'SD'
Columna 'Sexo': 6 veces que aparece 'SD'
Columna 'Edad': 0 veces que aparece 'SD'


### 4. Creacion Variables Adicionales

Consideramos la opción de generar nuevas características mediante la extracción de información valiosa de las columnas ya existentes.

#### Variable "Dia Semana" donde indicamos el dia de la semana donde sucedio el hecho

In [828]:
df_hechos["Fecha"].dtype

dtype('O')

In [829]:
df_hechos["DiaSemana"] = df_hechos["Fecha"].apply(lambda x: calendar.day_name[x.weekday()])
df_hechos.head(2)

,Id,Nro Victimas,Fecha,Año,Mes,Dia,Hora,Franja Horaria,Lugar,Tipo Calle,Calle,Cruce,Dirección Normalizada,Comuna,Longitud,Latitud,Participantes,Victima,Acusado,DiaSemana
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,PIEDRA BUENA AV.,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,-58.475340,-34.687570,MOTO-AUTO,MOTO,AUTO,Friday
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,AUTOPISTA,"PAZ, GRAL. AV.",DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,-58.508775,-34.669777,AUTO-PASAJEROS,AUTO,PASAJEROS,Saturday


In [830]:
traduccion_dias = {
    'Monday': 'Lunes',
    'Tuesday': 'Martes',
    'Wednesday': 'Miércoles',
    'Thursday': 'Jueves',
    'Friday': 'Viernes',
    'Saturday': 'Sábado',
    'Sunday': 'Domingo'
}

df_hechos["DiaSemana"] = df_hechos["DiaSemana"].map(traduccion_dias)
df_hechos.head(2)

,Id,Nro Victimas,Fecha,Año,Mes,Dia,Hora,Franja Horaria,Lugar,Tipo Calle,Calle,Cruce,Dirección Normalizada,Comuna,Longitud,Latitud,Participantes,Victima,Acusado,DiaSemana
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,PIEDRA BUENA AV.,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,-58.475340,-34.687570,MOTO-AUTO,MOTO,AUTO,Viernes
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,AUTOPISTA,"PAZ, GRAL. AV.",DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,-58.508775,-34.669777,AUTO-PASAJEROS,AUTO,PASAJEROS,Sábado


#### Variable "Momento del dia" en la que se indica en que momento del dia sucedio el hecho: mañana, tarde o noche

In [831]:
def asignar_momento_dia(hora):
    if pd.notnull(hora) and 6 <= hora.hour < 12:
        return "Mañana"
    elif pd.notnull(hora) and 12 <= hora.hour < 18:
        return "Tarde"
    elif pd.notnull(hora) and 18 <= hora.hour < 24:
        return "Noche"
    elif pd.notnull(hora):
        return "Madrugada"
    else:
        return "Sin Dato"  

In [832]:
df_hechos["MomentoDia"]= df_hechos["Hora"]. apply(asignar_momento_dia)
df_hechos[["Hora","MomentoDia"]].head(15)

,Hora,MomentoDia
0,04:00:00,Madrugada
1,01:15:00,Madrugada
2,07:00:00,Mañana
3,00:00:00,Madrugada
4,05:20:00,Madrugada
5,18:30:00,Noche
6,19:10:00,Noche
7,15:20:00,Tarde
8,01:20:00,Madrugada
9,11:30:00,Mañana


In [833]:
df_hechos.head()

,Id,Nro Victimas,Fecha,Año,Mes,Dia,Hora,Franja Horaria,Lugar,Tipo Calle,...,Cruce,Dirección Normalizada,Comuna,Longitud,Latitud,Participantes,Victima,Acusado,DiaSemana,MomentoDia
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,-58.475340,-34.687570,MOTO-AUTO,MOTO,AUTO,Viernes,Madrugada
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,AUTOPISTA,...,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,-58.508775,-34.669777,AUTO-PASAJEROS,AUTO,PASAJEROS,Sábado,Madrugada
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,Sin cruce,ENTRE RIOS AV. 2034,1,-58.390403,-34.631894,MOTO-AUTO,MOTO,AUTO,Domingo,Mañana
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,-58.465039,-34.680930,MOTO-SD,MOTO,SD,Domingo,Madrugada
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,...,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,-58.387183,-34.622466,MOTO-PASAJEROS,MOTO,PASAJEROS,Jueves,Madrugada


## 6. Unificacion de datasets

In [834]:
df_hechos.columns

Index(['Id', 'Nro Victimas', 'Fecha', 'Año', 'Mes', 'Dia', 'Hora',
       'Franja Horaria', 'Lugar', 'Tipo Calle', 'Calle', 'Cruce',
       'Dirección Normalizada', 'Comuna', 'Longitud', 'Latitud',
       'Participantes', 'Victima', 'Acusado', 'DiaSemana', 'MomentoDia'],
      dtype='object')

In [835]:
df_victimas.columns

Index(['Id', 'Fecha', 'Año', 'Mes', 'Dia', 'Rol', 'Victima', 'Sexo', 'Edad'], dtype='object')

Se eliminan columnas que se encuentran repetidas para que se haga la correcta union de los dataframes

In [836]:
df_victimas = df_victimas.drop(["Fecha","Año","Mes","Dia","Victima"], axis=1)
df_victimas.columns

Index(['Id', 'Rol', 'Sexo', 'Edad'], dtype='object')

In [837]:
df_siniestros = df_hechos.merge(df_victimas, on="Id", how="left")
df_siniestros.head(5)

,Id,Nro Victimas,Fecha,Año,Mes,Dia,Hora,Franja Horaria,Lugar,Tipo Calle,...,Longitud,Latitud,Participantes,Victima,Acusado,DiaSemana,MomentoDia,Rol,Sexo,Edad
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,-58.475340,-34.687570,MOTO-AUTO,MOTO,AUTO,Viernes,Madrugada,CONDUCTOR,MASCULINO,19.0
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,AUTOPISTA,...,-58.508775,-34.669777,AUTO-PASAJEROS,AUTO,PASAJEROS,Sábado,Madrugada,CONDUCTOR,MASCULINO,70.0
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,-58.390403,-34.631894,MOTO-AUTO,MOTO,AUTO,Domingo,Mañana,CONDUCTOR,MASCULINO,30.0
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,-58.465039,-34.680930,MOTO-SD,MOTO,SD,Domingo,Madrugada,CONDUCTOR,MASCULINO,18.0
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,...,-58.387183,-34.622466,MOTO-PASAJEROS,MOTO,PASAJEROS,Jueves,Madrugada,CONDUCTOR,MASCULINO,29.0


Verificamos los tipos de datos que tenemos y cantidad de nulos

In [838]:
df_siniestros.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 707 entries, 0 to 706
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   Id                     707 non-null    object  
 1   Nro Victimas           707 non-null    int64   
 2   Fecha                  707 non-null    object  
 3   Año                    707 non-null    int64   
 4   Mes                    707 non-null    int64   
 5   Dia                    707 non-null    int64   
 6   Hora                   707 non-null    object  
 7   Franja Horaria         707 non-null    int64   
 8   Lugar                  707 non-null    object  
 9   Tipo Calle             707 non-null    category
 10  Calle                  707 non-null    object  
 11  Cruce                  707 non-null    object  
 12  Dirección Normalizada  707 non-null    object  
 13  Comuna                 707 non-null    category
 14  Longitud               707 non-null    flo

In [839]:
columnas_sd_dicc3 = {}  

for columna3 in df_siniestros.columns:
    cantidad3 = (df_siniestros[columna3] == 'SD').sum()
    columnas_sd_dicc3[columna3] = cantidad3

for columna3, conteo3 in columnas_sd_dicc3.items():
    print(f"Columna '{columna3}': {conteo3} veces que aparece 'SD'")

Columna 'Id': 0 veces que aparece 'SD'
Columna 'Nro Victimas': 0 veces que aparece 'SD'
Columna 'Fecha': 0 veces que aparece 'SD'
Columna 'Año': 0 veces que aparece 'SD'
Columna 'Mes': 0 veces que aparece 'SD'
Columna 'Dia': 0 veces que aparece 'SD'
Columna 'Hora': 0 veces que aparece 'SD'
Columna 'Franja Horaria': 0 veces que aparece 'SD'
Columna 'Lugar': 0 veces que aparece 'SD'
Columna 'Tipo Calle': 0 veces que aparece 'SD'
Columna 'Calle': 0 veces que aparece 'SD'
Columna 'Cruce': 0 veces que aparece 'SD'
Columna 'Dirección Normalizada': 0 veces que aparece 'SD'
Columna 'Comuna': 0 veces que aparece 'SD'
Columna 'Longitud': 0 veces que aparece 'SD'
Columna 'Latitud': 0 veces que aparece 'SD'
Columna 'Participantes': 0 veces que aparece 'SD'
Columna 'Victima': 8 veces que aparece 'SD'
Columna 'Acusado': 20 veces que aparece 'SD'
Columna 'DiaSemana': 0 veces que aparece 'SD'
Columna 'MomentoDia': 0 veces que aparece 'SD'
Columna 'Rol': 0 veces que aparece 'SD'
Columna 'Sexo': 2 veces

In [840]:
df_siniestros.isnull().sum()

Id                       0
Nro Victimas             0
Fecha                    0
Año                      0
Mes                      0
Dia                      0
Hora                     0
Franja Horaria           0
Lugar                    0
Tipo Calle               0
Calle                    0
Cruce                    0
Dirección Normalizada    0
Comuna                   0
Longitud                 0
Latitud                  0
Participantes            0
Victima                  0
Acusado                  0
DiaSemana                0
MomentoDia               0
Rol                      0
Sexo                     0
Edad                     2
dtype: int64

## 7. Carga de datos

In [841]:
df_hechos.to_csv("Datasets/df_hechos.csv", index=False)
df_victimas.to_csv("Datasets/df_victimas.csv", index=False)
df_siniestros.to_csv("Datasets/df_siniestros.csv", index=False)